In [81]:
# Importing necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import median_absolute_error
from sklearn.datasets import fetch_openml
from sklearn.impute import SimpleImputer
import pandas as pd
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import BaggingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import learning_curve
import numpy as np
import cv2 as cv
from sklearn.preprocessing import MinMaxScaler

# Loading the Boston housing dataset
# housing = fetch_openml(name="house_prices", as_frame=True)



In [82]:
split_df =  pd.read_csv('split_df.csv',encoding='utf-8-sig')


In [83]:
split_df

,Table 1,Spectacles,Age,Gender,Name,id,Eye,VA,CDR,Diagnosis,IOP,Image
0,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,0,0.6,0.5,0,14.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
1,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,1,0.7,0.9,1,23.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
2,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,0,NaN,NaN,1,25.0,Data/SHAHEN_MERO Optic nerve.png_ right.png
3,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,1,9.0,0.8,1,40.0,Data/SHAHEN_MERO Optic nerve.png_ right.png
4,لا يوجد زرق,0.0,48,1,رندة ناصر,3.0,0,1.0,0.5,0,15.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
85,زرق أيسر تم معالجته جراحيا,1.0,55,1,زريفة المحمد,43.0,1,NaN,NaN,0,NaN,Data/BASEL_BASH Optic nerve.png_ left.png
86,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,0,0.3,0.5,1,15.0,Data/BASEL_BASH Optic nerve.png_ right.png
87,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,1,0.6,0.7,1,16.0,Data/BASEL_BASH Optic nerve.png_ right.png
88,لا يوجد زرق,1.0,65,0,حمود غشام,45.0,0,0.5,0.3,0,15.0,Data/NEZAR_HELOU Optic nerve.png_ left.png


In [84]:
# Example DataFrame 'data' with an 'Age' column
data = split_df['Age']
# Calculate quantiles to divide data into 3 groups
quantiles = [0, 1/3, 2/3, 1]

# Assign labels to the quantiles
labels = ['Low', 'Medium', 'High']

# Create a new column 'Age Group' based on the quantiles
split_df['Age Group'] = pd.qcut(data, q=quantiles, labels=labels)

In [85]:
# split_df.dropna(inplace=True)
sp = split_df.drop(['Table 1','Gender','Age','Spectacles','id'],axis=1)


In [86]:
sp = sp.drop(['Eye','Name'],axis=1)

In [87]:
sp.dropna(thresh=sp.shape[1]-1, inplace=True)


In [88]:
y_data = sp['Diagnosis']
X_data = sp.drop(['Diagnosis','Image'],axis=1)

In [89]:
X_data

,VA,CDR,IOP,Age Group
0,0.6,0.5,14.0,Medium
1,0.7,0.9,23.0,Medium
3,9.0,0.8,40.0,Medium
4,1.0,0.5,15.0,Low
5,1.0,0.4,14.0,Low
...,...,...,...,...
84,0.2,0.8,10.0,Medium
86,0.3,0.5,15.0,High
87,0.6,0.7,16.0,High
88,0.5,0.3,15.0,High


In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

vic = vectorizer.fit_transform(X_data['Age Group'])

In [91]:
for i, feature_name in enumerate(vectorizer.get_feature_names_out()):
    X_data[f'Vic_{feature_name}'] = vic[:, i].toarray().ravel()

In [92]:
X_data


,VA,CDR,IOP,Age Group,Vic_high,Vic_low,Vic_medium
0,0.6,0.5,14.0,Medium,0.0,0.0,1.0
1,0.7,0.9,23.0,Medium,0.0,0.0,1.0
3,9.0,0.8,40.0,Medium,0.0,0.0,1.0
4,1.0,0.5,15.0,Low,0.0,1.0,0.0
5,1.0,0.4,14.0,Low,0.0,1.0,0.0
...,...,...,...,...,...,...,...
84,0.2,0.8,10.0,Medium,0.0,0.0,1.0
86,0.3,0.5,15.0,High,1.0,0.0,0.0
87,0.6,0.7,16.0,High,1.0,0.0,0.0
88,0.5,0.3,15.0,High,1.0,0.0,0.0


In [93]:
X_data.drop('Age Group',axis=1,inplace=True)

In [94]:

pipeline = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    StandardScaler(),
    # SelectKBest(f_classif, k=3),  # Select the top 5 features based on ANOVA F-value
    BaggingClassifier(
        KNeighborsClassifier(n_neighbors=3, weights='uniform', algorithm='auto', p=2),  # p=2 for Euclidean distance (L2)
        n_estimators=10,  # Number of base estimators in the ensemble
        random_state=42
    )
)

In [95]:
from sklearn import svm


clf = svm.SVC(C=0.9,kernel='linear')


In [96]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(random_state=42)


In [97]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.1, random_state=44, shuffle=True)


In [98]:
y_train

41    0
40    0
66    0
53    0
20    1
     ..
4     0
60    0
46    1
36    1
21    1
Name: Diagnosis, Length: 79, dtype: int64

In [99]:
y_test

89    0
83    0
34    1
81    1
8     1
25    1
69    1
71    1
61    0
Name: Diagnosis, dtype: int64

In [100]:
imputer = SimpleImputer(strategy='most_frequent')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)


# scaler = MinMaxScaler()
# X_train_imputed = scaler.fit_transform(X_train)
# X_test_imputed = scaler.transform(X_test)

In [101]:
X_test_imputed.shape

(9, 6)

In [102]:
y_train.shape

(79,)

In [103]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('KNN', pipeline), ('SVM', clf), ('DT', decision_tree)],
    voting='hard')
voting_clf.fit(X_train_imputed, y_train)

VotingClassifier(estimators=[('KNN',
                              Pipeline(steps=[('simpleimputer',
                                               SimpleImputer(strategy='most_frequent')),
                                              ('standardscaler',
                                               StandardScaler()),
                                              ('baggingclassifier',
                                               BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=3),
                                                                 random_state=42))])),
                             ('SVM', SVC(C=0.9, kernel='linear')),
                             ('DT', DecisionTreeClassifier(random_state=42))])

In [104]:
voting_clf.score(X_train_imputed,y_train)


0.9113924050632911

In [105]:
from sklearn.metrics import accuracy_score

for clf in (pipeline, clf, decision_tree, voting_clf):
    clf.fit(X_train_imputed, y_train)
    y_pred = clf.predict(X_test_imputed)
    #print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
    print('Accuracy of {} classifier on training set: {:.2f}'
     .format(clf.__class__.__name__,clf.score(X_train_imputed, y_train)))
    print('Accuracy of {} classifier on test set: {:.2f}'
     .format(clf.__class__.__name__,clf.score(X_test_imputed, y_test)))
    print()

Accuracy of Pipeline classifier on training set: 0.90
Accuracy of Pipeline classifier on test set: 0.89

Accuracy of SVC classifier on training set: 0.72
Accuracy of SVC classifier on test set: 0.89

Accuracy of DecisionTreeClassifier classifier on training set: 0.97
Accuracy of DecisionTreeClassifier classifier on test set: 0.78

Accuracy of VotingClassifier classifier on training set: 0.91
Accuracy of VotingClassifier classifier on test set: 0.89



In [108]:
def predict_category(s, train=X_train_imputed, model=voting_clf, victorizer=vectorizer):
    vic_sent = victorizer.transform([s[3]]).toarray()
    # arr = [s[0],s[1],s[2],vic_sent]
    num_features = np.array(s[:3]).reshape(1, -1)
    arr = np.hstack([num_features, vic_sent])
    print(arr)
    pred = model.predict(arr)
    return pred[0]

In [109]:
predict_category([0.6,0.5,14.0,'Medium'])

[[ 0.6  0.5 14.   0.   0.   1. ]]


1